In [ ]:
# BLOCK 1 : FIND THE PATH OF THE TROPICAL CYCLONE DATA THAT WAS DOWNLOADER OVER WIS2
import os
import glob

def find_latest_file_with_path(root_dir, sub_path, extension):
    # Construct the search pattern to include the specific path and extension
    search_pattern = os.path.join(root_dir, '**', sub_path, f'*.{extension}')
    # Use glob to find all matching files recursively
    matching_files = glob.glob(search_pattern, recursive=True)
    if not matching_files:
        return None
    # Find the latest file based on the modification time
    latest_file = max(matching_files, key=os.path.getmtime) 
    return latest_file

# Set the root directory and the specific sub-path
root_dir = '/root/downloads'
sub_path = 'forecast/medium-range/deterministic/trajectory'
extension = 'bin'

# Call the function
latest_file = find_latest_file_with_path(root_dir, sub_path, extension)

if latest_file:
    print(f"The latest .bin file with the path '{sub_path}' is: {latest_file}")
else:
    print(f"No .bin files found with the path '{sub_path}'.")

In [ ]:
# BLOCK 2 : PARSE THE DATA 
infile = latest_file

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import LAND, OCEAN, COASTLINE
from datetime import datetime as dt, timedelta
from eccodes import codes_bufr_new_from_file, codes_set, codes_get, codes_get_array, codes_get_double, codes_get_long
from eccodes import CODES_MISSING_DOUBLE
from functools import partial
import json
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import colormaps
import numpy as np
import pyproj
from shapely.geometry import shape, Point, MultiPoint
from shapely.ops import transform

# set map bounds, this will be updated as part of reading the data
bounds = [180,-180, 90, -90]


# open and create bufr handle
fh = open(infile,"rb")
bufr = codes_bufr_new_from_file(fh)
# unpack
codes_set(bufr,"unpack",True)
compressed = codes_get(bufr, 'compressedData')

# get storm identifier
stormIdentifier = codes_get(bufr, "stormIdentifier")

# get number of subsets
nSubsets = codes_get(bufr,"numberOfSubsets")
print(nSubsets)
# get date and time of T0
year = codes_get(bufr,"#1#year")
month = codes_get(bufr,"#1#month")
day = codes_get(bufr,"#1#day")
hour = codes_get(bufr,"#1#hour")
minute = codes_get(bufr,"#1#minute")
t0 = dt(year=year, month=month, day=day, hour=hour, minute=minute)
# get location of low pressure centre at t0
lon0 = [float(val) for val in codes_get_array(bufr,"#2#longitude")]
if compressed and len(lon0) == 1 :
    lon0 = lon0 * nSubsets
lat0 = [float(val) for val in codes_get_array(bufr,"#2#latitude")]
if compressed and len(lat0) == 1 :
    lat0 = lat0 * nSubsets
# and the pressure
slp0 = [float(val) for val in codes_get_array(bufr,"#1#pressureReducedToMeanSeaLevel")]
if compressed and len(slp0) == 1:
    slp0 = slp0*nSubsets
# get time periods
timePeriods = [float(val) for val in codes_get_array(bufr,"timePeriod")]
numberTimesteps = len(timePeriods)

# now extract a track for each subset
objs = []
deterministic = {
    "type": "Feature",
    "geometry": {
        "type": "MultiPoint",
        "coordinates": [ ]
    },
    "properties": {
        "stormIdentifier": stormIdentifier,
        "datetime": [ ],
        "slp": [ ]
    }        
}
subset = nSubsets - 1 # last ensemble member is deterministic
if lat0[subset] == CODES_MISSING_DOUBLE or lon0[subset] == CODES_MISSING_DOUBLE or slp0[subset] == CODES_MISSING_DOUBLE:
        lat0[subset] = None
        lon0[subset] = None
        slp0[subset] = None
else:
    deterministic['geometry']['coordinates'].append([lon0[subset], lat0[subset]])
    deterministic['properties']['datetime'].append(t0)
    deterministic['properties']['slp'].append(slp0[subset])
    bounds[0] = min(bounds[0], lon0[subset])
    bounds[1] = max(bounds[1], lon0[subset])
    bounds[2] = min(bounds[2], lat0[subset])
    bounds[3] = max(bounds[3], lat0[subset])

for subset in range(nSubsets-2):
    objs.append( {
            "type": "Feature",
            "geometry": {
                "type": "MultiPoint",
                "coordinates": []
            },
            "properties": {
                "stormIdentifier": stormIdentifier,
                "ensembleMember": subset,
                "datetime": [],
                "slp": []
            }        
        })    
    if lat0[subset] == CODES_MISSING_DOUBLE or lon0[subset] == CODES_MISSING_DOUBLE or slp0[subset] == CODES_MISSING_DOUBLE:
        lat0[subset] = None
        lon0[subset] = None
        slp0[subset] = None
        continue
    objs[subset]['geometry']['coordinates'].append([lon0[subset], lat0[subset]])
    objs[subset]['properties']['datetime'].append(t0)
    objs[subset]['properties']['slp'].append(slp0[subset])
    bounds[0] = min(bounds[0], lon0[subset])
    bounds[1] = max(bounds[1], lon0[subset])
    bounds[2] = min(bounds[2], lat0[subset])
    bounds[3] = max(bounds[3], lat0[subset])
    
t = t0   
for idx in range(numberTimesteps):
    indexNumber = (idx+1)*2 + 2    
    lat = [float(val) for val in codes_get_array(bufr, f"#{indexNumber:d}#latitude")]
    lon = [float(val) for val in codes_get_array(bufr, f"#{indexNumber:d}#longitude")]
    slp = [float(val) for val in codes_get_array(bufr, f"#{idx+2}#pressureReducedToMeanSeaLevel")]
    t = t0 + timedelta(hours=int(timePeriods[idx]))
    for subset in range(nSubsets-2):        
        if lat[subset] == CODES_MISSING_DOUBLE or lon[subset] == CODES_MISSING_DOUBLE or slp[subset] == CODES_MISSING_DOUBLE:
            lat[subset] = None
            lon[subset] = None
            slp[subset] = None
        else:
            objs[subset]['geometry']['coordinates'].append(  [lon[subset], lat[subset]]  )
            objs[subset]['properties']['datetime'].append(t)
            objs[subset]['properties']['slp'].append(slp[subset])
            bounds[0] = min(bounds[0], lon[subset])
            bounds[1] = max(bounds[1], lon[subset])
            bounds[2] = min(bounds[2], lat[subset])
            bounds[3] = max(bounds[3], lat[subset])

    subset = nSubsets - 1 # last ensemble member is deterministic
    if lat[subset] == CODES_MISSING_DOUBLE or lon[subset] == CODES_MISSING_DOUBLE or slp[subset] == CODES_MISSING_DOUBLE:
            lat[subset] = None
            lon[subset] = None
            slp[subset] = None
    else:
        deterministic['geometry']['coordinates'].append([lon[subset], lat[subset]])
        deterministic['properties']['datetime'].append(t)
        deterministic['properties']['slp'].append(slp[subset])
        bounds[0] = min(bounds[0], lon[subset])
        bounds[1] = max(bounds[1], lon[subset])
        bounds[2] = min(bounds[2], lat[subset])
        bounds[3] = max(bounds[3], lat[subset])

In [ ]:
# BLOCK 3: CALCULATE STRIKE PROBABILITIES AND CREATE A MAP
# NOTE: basic calculation showm, this needs to be checked

# First function to create a grid
def create_grid(bounds, resolution): 
    lons = np.arange(bounds[0], bounds[1] + resolution, resolution) 
    lats = np.arange(bounds[2], bounds[3] + resolution, resolution) 
    return np.meshgrid(lons, lats)

# Next function to calculate strike probabilities
def calculate_track_probabilities(tracks, grid_lons, grid_lats, distance_threshold):
    # Create a geodetic distance calculator
    geod = pyproj.Geod(ellps='WGS84')
    # Create transformer objects
    wgs84 = pyproj.CRS('EPSG:4326')
    web_mercator = pyproj.CRS('EPSG:3857')
    project = pyproj.Transformer.from_crs(wgs84, web_mercator, always_xy=True).transform
    project_back = pyproj.Transformer.from_crs(web_mercator, wgs84, always_xy=True).transform
    
    # Function to create a buffer around a point
    def point_buffer(lon, lat):
        point = Point(lon, lat)
        point_projected = transform(project, point)
        buffer = point_projected.buffer(distance_threshold)
        return transform(project_back, buffer)

    probabilities = np.zeros_like(grid_lons)
    total_tracks = len(tracks)

    for i in range(grid_lons.shape[0]):
        for j in range(grid_lons.shape[1]):
            grid_point = Point(grid_lons[i, j], grid_lats[i, j])
            buffer = point_buffer(grid_lons[i, j], grid_lats[i, j])            
            tracks_within_buffer = sum(1 for track in tracks if buffer.intersects(track))
            probabilities[i, j] = tracks_within_buffer / total_tracks

    return probabilities

# Now do the calculation
tracks = [MultiPoint(feature['geometry']['coordinates']) for feature in objs]
resolution = 1
grid_lons, grid_lats = create_grid(np.add(bounds, [-5, 5, -5, 5]), resolution)
distance_threshold = 200000
probabilities = calculate_track_probabilities(tracks, grid_lons, grid_lats, distance_threshold)

# now a new map showing the strike probabilities
fig, ax = plt.subplots(figsize=(18, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Set extent with 5 degree buffer
ax.set_extent( np.add(bounds,[-5,5,-5,5]), crs=ccrs.PlateCarree() )

# Add land, coastlines, etc.
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Overlay probabilities
cs = ax.contourf(grid_lons, grid_lats, probabilities, levels=10, cmap='YlOrRd', transform=ccrs.PlateCarree(), alpha=0.7)

# Add colorbar
cbar = plt.colorbar(cs, ax=ax, orientation='vertical', pad=0.1)
cbar.set_label('Probability')

# Add title and labels
plt.title(f'Probability of tropical storm track within 200km\n{t0} - {t}', fontsize=16)
plt.text(bounds[0]-4.5, bounds[3]+3, f"Storm: {stormIdentifier}")
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Add gridlines
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add deterministic track and pressures at midnight
ax.plot(lon, lat, marker="o", transform=ccrs.PlateCarree())
for idx in range(len(lon)):
    if deterministic['properties']['datetime'][idx].hour == 0:
        plt.text(lon[idx], lat[idx]+1,deterministic['properties']['datetime'][idx].date(), horizontalalignment='center', transform=ccrs.PlateCarree() )
        plt.text(lon[idx], lat[idx]-1,deterministic['properties']['slp'][idx]/100.0, horizontalalignment='center', verticalalignment='bottom',transform=ccrs.PlateCarree() )
# now show
plt.show()